# Flow over Cylinder Case

In [ ]:
from meshpy import triangle, geometry
import numpy as np
import jax.numpy as jnp
import matplotlib.pyplot as plt
from tqdm import tqdm
import sys
sys.path.append("..")
from src.mesher import *
from src.environment import *
from src.dynamics import *
from src.refiner import *

## Building Geometry

In [ ]:
builder = geometry.GeometryBuilder()

In [ ]:
points, facets, _, _ = geometry.make_box((0, 0), (6.0, 3.0))
builder.add_geometry(points, facets, facet_markers=[1, 2, 3, 4])

In [ ]:
points, facets, _, facet_markers = geometry.make_circle(.25, (1.5, 1.5), 100, 5)
builder.add_geometry(points, facets, facet_markers=facet_markers)

In [ ]:
for facet, marker in zip(builder.facets, builder.facet_markers):
    match marker:
        case 1:
            color = 'blue'
        case 2:
            color = 'green'
        case 3:
            color = 'orange'
        case 4:
            color = 'purple'
    plt.plot(np.asarray(builder.points)[facet, 0], np.asarray(builder.points)[facet, 1], color=color, linewidth=0.5)
    plt.text(np.mean(np.asarray(builder.points)[facet, 0]), np.mean(np.asarray(builder.points)[facet, 1]), str(marker), fontsize=8, ha='center')
plt.axis('equal')
plt.show()


In [ ]:
info = triangle.MeshInfo()
builder.set(info)
info.set_holes([(1.5,1.5)])

## Building Mesh

In [ ]:
mesh = triangle.build(info,
                      min_angle=33.0,
                      max_volume=.0005,
                      generate_faces=True,
                      volume_constraints=True,)
np.array(mesh.elements).shape[0]

In [ ]:
improver = MeshRefiner(mesh,[(1.5,1.5)])
improver.show_mesh_quality()
mesh = improver.improve(aspect_thresh=3.0,
                        skew_thresh=30.0,
                        max_volume=0.0005,
                        move_fraction=.25,
                        max_iter=10)
improver.show_mesh_quality()

In [ ]:
mesh_points = np.array(mesh.points)
mesh_tris = np.array(mesh.elements)
mesh_attr = np.array(mesh.point_markers)

import matplotlib.pyplot as plt

plt.triplot(mesh_points[:, 0], mesh_points[:, 1], mesh_tris)
plt.xlabel("x")
plt.ylabel("y")
#
n = np.size(mesh_attr)
inner_nodes = [i for i in range(n) if mesh_attr[i] == 1]
outer_nodes = [i for i in range(n) if mesh_attr[i] == 3]
plt.plot(mesh_points[inner_nodes, 0], mesh_points[inner_nodes, 1], "ro")
plt.plot(mesh_points[outer_nodes, 0], mesh_points[outer_nodes, 1], "go")
plt.axis("equal")
plt.axis([.75,1.25,.75,1.25])
plt.show()

In [ ]:
mesher = Mesher()
mesher.import_meshpy(mesh)
mesher.calc_mesh_properties()
mesher.verify_stencil_geometry()

## Define Conditions

In [ ]:
# Given physical parameters
Re = 100            # Reynolds number
nu = 0.05     # physical kinematic viscosity
L = 50             # circ length
dx = 1           # spatial step
dt = .1            # time step
rho = .95          # density

# Speed of sound squared for D2Q9 lattice
c_s_sq = 1.0 / 3.0

# Convert to lattice units
U_lattice = Re * nu / L

# Compute relaxation time tau
Tau = nu / c_s_sq + 0.5

# Print results
print(f"Inlet velocity (lattice units): {U_lattice:.5f}")
print(f"Relaxation time tau: {Tau:.5f}")
U_lid = U_lattice
dynamics = D2Q9(tau=Tau,delta_t = dt)

In [ ]:
cells,faces,nodes = mesher.to_env(dynamics,flux_method="upwind")
nodes = mesher.set_vel_node(nodes,marker = 4, velocity=jnp.array([U_lid, 0.0]))
nodes = mesher.set_vel_node(nodes,marker = 3, velocity=jnp.array([0.0, 0.0]))
nodes = mesher.set_vel_node(nodes,marker = 1, velocity=jnp.array([0.0, 0.0]))
nodes = mesher.set_vel_node(nodes,marker = 5, velocity=jnp.array([0.0, 0.0]))
nodes = mesher.set_rho_node(nodes,marker = 2, rho=rho)

In [ ]:
env = Environment(cells,faces,nodes)
env.init()

In [ ]:
for i in tqdm(range(100000)):
    env = env.step()

In [ ]:
xy = np.array(mesher.cell_centers)
x = xy[:,0]
y = xy[:,1]
vel = env.cells.vel 
mag = np.sqrt(np.sum(vel**2,axis=-1))
plt.quiver(x,y,vel[:,0],vel[:,1],mag,scale=10)
xy = np.asarray(mesher.points)
x = xy[:,0]
y = xy[:,1]
tri = np.asarray(mesher.cells)
plt.triplot(x,y,tri,linewidth=0.5)
plt.axis([1.2,1.8,1.2,1.8])
plt.colorbar()
plt.show()

In [ ]:
xy = np.array(mesher.cell_centers)
x = xy[:,0]
y = xy[:,1]
vel = env.cells.vel
mag = np.sqrt(np.sum(vel**2,axis=-1))
plt.scatter(x, y, c=mag, cmap='viridis', vmin=jnp.min(mag), vmax=jnp.max(mag), s=1, marker='o')
plt.axis('equal')
plt.colorbar()
plt.show()

In [ ]:
xy = np.array(mesher.cell_centers)
x = xy[:,0]
y = xy[:,1]
rho = env.cells.rho
plt.scatter(x, y, c=rho, cmap='viridis', vmin=jnp.min(rho), vmax=jnp.max(rho), s=1, marker='o')
plt.axis('equal')
plt.colorbar()
plt.show()